# **Taller #2**

Integrantes :
*   Maria Fernanda Díaz Velazquez
*   Fabio Ordoñez
*   Juan Pablo Pinzón González



In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from scipy.optimize import linprog
!pip install yfinance
import yfinance as yf
import cvxpy as cp
from scipy import stats
import warnings
warnings.filterwarnings("ignore")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 KB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 KB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.8/112.8 KB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.2/112.2 KB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 KB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 9.7 MB/s eta 0:00:00
  Attempting uninstall: html5lib
    Found existing installation: html5lib 1.0.1
    Uninstalling html5lib-1.0.1:
      Successfully uninstalled html5lib-1.0.1
  Attempting uninstall: requests
    Found existing installation: requests 2.25.1
    Uninstalling requests-2.25.1:
      Successfully uninstalled requests-2.25.1
  Attempting un

1. La empresa XYZ produce dos productos: A y B. La ganancia por unidad de A es de $3 dólares y de B es de $5 dólares. Para su producción dispone de 500 unidades de materia prima, 1000 unidades de mano de obra y 2000 dólares de capital, de los cuales se necesitan para producir una unidad de:

i) A: una unidad de materia prima, dos unidades de mano de obra y una unidad de capital.

ii) B: una unidad de materia prima, cuatro unidades de mano de obra y dos unidades de
capital.

Además, se necesita producir al menos 100 unidades de A y 150 unidades de B para
satisfacer la demanda del mercado.
Encuentre la asignación óptima de A y B que maximizan las ganancias de la empresa. Para ello, formule un problema de LP utilizando Scipy.

In [2]:
from scipy.optimize import linprog

# Coeficientes de la función objetivo
c = [-3, -5]

# Coeficientes de las restricciones
A = [[1, 1], [2, 4], [1, 2]]
b = [500, 1000, 2000]

# Restricciones de no negatividad
x0_bounds = (100, None)
x1_bounds = (150, None)

# Resolviendo el problema de programación lineal
res = linprog(c, A_ub=A, b_ub=b, bounds=[x0_bounds, x1_bounds], method='simplex')

# Imprimiendo los resultados
print("Producción óptima de A:", round(res.x[0], 2))
print("Producción óptima de B:", round(res.x[1], 2))
print("Ganancia máxima: ${:,.2f}".format(-res.fun))

Producción óptima de A: 200.0
Producción óptima de B: 150.0
Ganancia máxima: $1,350.00


2. Usted desea invertir la suma de $100 millones en cuatro fondos que tienen diferentes
retornos esperados (E(R)), así como diferentes ponderaciones en acciones de alto valor,
gran tamaño y alta liquidez, como muestra la siguiente tabla:

In [3]:
#En linprog

c = np.array([0.1,0.15,0.16,0.08])
A = np.array([[1,1,1,1]])
b = np.array([100])

D = np.array([[0.5,0.3,0.25,0.6],
              [0.3,0.1,0.4,0.2],
              [0.2,0.6,0.35,0.2]])
d = np.array([35,30,15])

resultado = linprog(-c,A_ub=-D,b_ub=-d,A_eq=A,b_eq=b, method='simplex')
resultado


 message: Optimization terminated successfully.
 success: True
  status: 0
     fun: -13.736842105263158
       x: [ 0.000e+00  1.579e+01  5.789e+01  2.632e+01]
     nit: 7

In [4]:
#CVXPY
n = len(c) 
x = cp.Variable(n) 

objective = cp.Maximize(c.T @ x )
constraints = [A @ x == b, D @x>=d,x>=0]
prob = cp.Problem(objective, constraints)

result = prob.solve()

sol = round(pd.DataFrame(x.value),4)
sol
#Interpretado como cantidad invertida

,0
0,0.0000
1,15.7895
2,57.8947
3,26.3158


3. La empresa XYZ se enfrente al siguiente problema de manejo de efectivo. Las cifras
están expresadas en millones ($).

Además, se sabe que la empresa cuenta con las siguientes fuentes de recursos:

• Una línea de crédito de hasta $100 a una tasa de interés del 1% mensual.

• Puede emitir papel comercial a 90 días con un interés total del 2% por un período
de 3 meses.

• Todos los meses se pueden invertir los fondos sobrantes a una tasa de interés del
0,3% mensual. Por ejemplo, la empresa podría hacer un depósito bancario que le
reconoce esa tasa.

Sobre este problema, la empresa desea saber si: ¿usar la línea de crédito en algunos de los meses le representa un alto costo? si es así ¿cuándo debería usarla? ¿cuánto cuesta? ¿qué pagos de intereses tendrá que hacer entre enero y junio? etc... Para responder a estos interrogantes, la empresa puede formular y resolver un problema de LP. Para la formulación del problema tener en cuenta lo siguiente:

In [5]:
# Datos del problema
FCneto = [-150, -100, 200, -200, 50, 300]

# Variables
x = cp.Variable(5, nonneg=True)  # Monto dispuesto a pedir en la línea de crédito en el mes j, j=1,2,3,4,5
y = cp.Variable((3,1), nonneg=True)  # Monto emitido en el papel comercial en el mes j, j=1,2,3
z = cp.Variable(6, nonneg=True)  # Saldo de fondos en el mes j, j=1,2,3,4,5,6

# Restricciones
constraints = []
constraints.append(x <= 100)  # xj <= 100 para j = 1,2,3,4,5
constraints.append(x >= 0)  # xj >= 0 para j = 1,2,3,4,5
constraints.append(y >= 0)  # yj >= 0 para j = 1,2,3
constraints.append(z >= 0)  # zj >= 0 para j = 1,2,3,4,5
constraints.append(x[0] + y[0] - z[0] == 150)
constraints.append(x[1] + y[1] - 1.01*x[0] + 1.003*z[0] - z[1] == 100)
constraints.append(x[2] + y[2] - 1.01*x[1] + 1.003*z[1] - z[2] == -200)
constraints.append(x[3] - 1.02*y[0] - 1.01*x[2] + 1.003*z[2] - z[3] == 200)
constraints.append(x[4] - 1.02*y[1] - 1.01*x[3] + 1.003*z[3] - z[4] == -50)
constraints.append(-1.02*y[2] - 1.01*x[4] + 1.003*z[4] - z[5] == -300)

# Función objetivo
obj = cp.Maximize(z[5])  # Maximizar el saldo de los fondos en el mes 6 (Junio)

# Resolución del problema
prob = cp.Problem(obj, constraints)
prob.solve()

# Resultados
print("Status de la solución:", prob.status)
print("Valor óptimo de la función objetivo:", prob.value)
print("Valor óptimo de las variables x:", x.value)
print("Valor óptimo de las variables y:", y.value)
print("Valor óptimo de las variables z:", z.value)


Status de la solución: optimal
Valor óptimo de la función objetivo: 92.49694913918181
Valor óptimo de las variables x: [1.52279431e-07 2.42871680e+01 4.07136341e-07 1.37937716e-07
 2.72270876e+01]
Valor óptimo de las variables y: [[150.00000077]
 [ 75.7128314 ]
 [176.47420844]]
Valor óptimo de las variables z: [9.18655879e-07 1.92601707e-07 3.51944169e+02 7.80918648e-07
 2.04373310e-07 9.24969491e+01]


4. La empresa XYZ desea encontrar el saldo óptimo que debe mantener en su cuenta
bancaria en un horizonte de un mes buscando maximizar su rentabilidad total. Se sabe que la empresa espera recibir entradas de efectivo de: $500 en la primera semana, $300 en la segunda semana, $200 en la tercera semana y $400 en la cuarta semana. Todas las cifras se encuentran en millones de pesos. Además, la empresa debe realizar los siguientes pagos durante el mes: $350 en la primera semana, $400 en la segunda semana, $250 en la tercera semana y $200 en la cuarta semana y, para ello, debe mantener un saldo suficiente que respalde estas obligaciones.

Para aprovechar la rentabilidad que ofrecen los saldos en cada momento, la empresa puede invertir sus recursos en un instrumento a corto plazo en el que recibe un rendimiento del 0,02% por día, pero al realizar esta operación debe incurrir en un costo de transacción del 0,01% del monto transferido.

Formule un problema de LP utilizando Scipy que le permita maximizar la rentabilidad total de los saldos en su cuenta teniendo en cuenta las restricciones de sus flujos de efectivo.

In [6]:
#Funcion objetivo
FO = np.array([0,0,0,0,0,0,0,1])

#Restricciones
Rn_4=(1+0.0001)**7
A_4= np.array([[1,0,0,0,-1,0,0,0],
                [0,1,0,0,Rn_4,-1,0,0],
                [0,0,1,0,0,Rn_4,-1,0],
                [0,0,0,1,0,0,Rn_4,-1]])
b_4= np.array([150,-100,-50,200])

#Funcion de optimización
Rta_pt4= linprog(-FO,A_ub=A_4,b_ub=-b_4,method="simplex")
Val_opti=pd.DataFrame(Rta_pt4.x).T
Val_opti

,0,1,2,3,4,5,6,7
0,0.0,0.0,0.0,0.0,150.0,50.105032,0.140116,200.140214


Cantidades que quedan ahi una vez que salen de la accion,maximizamos

5. La empresa XYZ tiene que decidir como distribuye su presupuesto 10.000 millones y la mano de obra (horas de trabajo) en tres departamentos: ventas, marketing y producción buscando maximizar sus ganancias. Para ello, se sabe que cada departamento le proporciona a la empresa las siguientes ganancias: ventas: $5, marketing: 8$, y producción: $10. Esta asignación debe cumplir las siguientes restricciones:

i. El total de horas de trabajo disponibles es de 1.000.

ii. El departamento de ventas debe recibir al menos el 20% del presupuesto total.

iii. El departamento de marketing debe recibir al menos el 30% del total de horas de
trabajo disponibles.

iv. El departamento de producción debe recibir al menos el 25% del total de horas de trabajo disponibles.

In [7]:
c=np.array([0,0,0,5,8,10])
A=np.array([[0,0,0,1,1,1],[1,1,1,0,0,0]])
D=np.array([[0,0,0,0,1,0],[0,1,0,0,0,0],[0,0,1,0,0,0],[0,0,0,1,0,0],[0,0,0,0,0,1]])
d=np.array([2000,300,250,2000,2000])
b=np.array([10000,1000])

rta=linprog(-c,A_ub=-D,b_ub=-d,A_eq=A,b_eq=b,method="simplex")
pd.DataFrame(rta.x).round(4)

,0
0,450.0
1,300.0
2,250.0
3,2000.0
4,2000.0
5,6000.0


6. Considere una empresa con tres puntos potenciales para ubicar sus instalaciones (J =
{1,2,3}) y cinco puntos de demanda que debe satisfacer (I = {1,2,3,4,5}). La apertura del
punto j ∈ J tiene un costo fijo de funcionamiento fj, independientemente del volumen de ventas que maneje. Este volumen está limitado a una determinada cantidad máxima que puede manejarse anualmente (Mj). Además, hay un costo de transporte cij para ir desde el punto j para abastecer la demanda en el punto i. Las siguientes tablas muestran los datos de demanda en cada punto:

In [8]:
demand = np.array([80, 270, 250, 160, 180])

costs = np.array([[4, 6, 9], [5, 4, 7], [6, 3, 4], [8, 5, 3], [10, 8, 4]])

fixed_costs = np.array([1000, 1000, 1000])

max_capacities = np.array([500, 500, 500])

In [9]:
x = cp.Variable((5, 3), nonneg=True)
y = cp.Variable(3, boolean=True)

objective = cp.Minimize(cp.sum(cp.multiply(fixed_costs, y)) + cp.sum(cp.multiply(costs, x)))

In [10]:
constraints = []

for i in range(5):
  constraints.append(cp.sum(x[i, :]) >= demand[i])

for j in range(3):
  constraints.append(cp.sum(x[:, j]) <= max_capacities[j] * y[j])

problem = cp.Problem(objective, constraints)
problem.solve()

5610.0

In [11]:
print("Optimal value:", problem.value)
print("x:\n", x.value)
print("y:", y.value)

Optimal value: 5610.0
x:
 [[  0.  80.  -0.]
 [ -0. 270.  -0.]
 [ -0. 150. 100.]
 [ -0.  -0. 160.]
 [ -0.  -0. 180.]]
y: [0. 1. 1.]
